In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

# Alpha development guidelines

### Alpha structure 
Each alpha should implement 3 main methods:
- **setup()**
In this method you should implement quotes fetching and commissions settings, and any preparations for alpha calculations.

- **calculate()**
This is main alpha calculation method, this method must return alpha exposure dataframe, you can use self.exposure() helper method to produce exposure from entry/exit rules, or make it by your own.

- **calculate_position()**
This method used for position construction based on exposure information returned from calculate(), here you can initiate (replicate) EXO index position or setup any custom position you want.

In [ ]:
from tmqrfeed.manager import DataManager
from tmqrfeed.quotes.quote_contfut import QuoteContFut
from tmqrfeed.costs import Costs
from datetime import datetime
import pandas as pd
from tmqrstrategy import StrategyAlpha
from tmqrstrategy.optimizers import OptimizerBase, OptimizerGenetic



def CrossUp(a, b):
    """
    A crosses up B
    """
    return (a.shift(1) < b.shift(1)) & (a > b)


def CrossDown(a, b):
    """
    A crosses down B
    """
    return (a.shift(1) > b.shift(1)) & (a < b)


class AlphaSampleHowto(StrategyAlpha):
    def __init__(self, datamanager: DataManager, **kwargs):
        
        # Call parent class __init__ this is always REQUIRED
        super().__init__(datamanager, **kwargs)
        
        # Set some internal alphas parameters if it's required
        self.temp = datetime.now()

    def setup(self):
        # Set the primary quotes in this case this is ContinousFutures US.ES EOD
        self.dm.series_primary_set(QuoteContFut, 'US.ES', timeframe='D')
        
        # Set secondary series for alpha
        # For example used same continuous futures series, but any Quote algo can be used
        # for example EXO index
        self.dm.series_extra_set('CONTFUT', QuoteContFut, 'US.ES', timeframe='D')
        
        # Set the costs
        # COSTS ARE REQUIRED
        #self.dm.costs_set('US', Costs()) # This sets zero costs
        self.dm.costs_set('US', Costs(per_option=3.0,
                                      per_contract=3.0)) # This 3.0 / 3.0 per future and options

    def calculate(self, *args):
        """
        Calculate main alpha logic
        """
        direction = 1
        
        ## *args - is a list of values stored in 'wfo_opt_params' of alpha's context
        
        # 1-st way to fetch params
        period_slow, period_fast = args
        
        # 2-nd way to fetch params
        period_slow = args[0]
        period_fast = args[1]

        
        # self.dm.quotes() - returs pandas.DataFrame with ['o','h','l','c','v'] columns
        # Colums names may be diffenernt depending of Quote* algo used in 'def setup(self)' method
        
        px = self.dm.quotes()['c']
        
        #
        # Fetching extra series set in self.setup()
        # 
        extra_ohlc = self.dm.quotes('CONTFUT')

        #
        #
        # Indicator calculation
        #
        #
        slow_ma = px.rolling(period_slow).mean()
        fast_ma = px.rolling(period_fast).mean()

        # Enry/exit rules
        entry_rule = CrossDown(fast_ma, slow_ma)
        exit_rule = (CrossUp(fast_ma, slow_ma))
        
        #
        # IMPORTANT:
        #   each alpha should return exposure dataframe (i.e. opened volume of position)
        #   - you can use own exposure calculation
        #   or
        #   - you can use self.exposure() shortcut (refer to the docstrings in the source code) 
        return self.exposure(entry_rule, exit_rule, direction)

    def calculate_position(self, date: datetime, exposure_record: pd.DataFrame):
        """
        This method used for position construction based on exposure information returned from calculate(), 
        here you can initiate (replicate) EXO index position or setup any custom position you want.
        """
        # Get the position of Quote algo (in this case current cont futures)
        primary_quotes_position = self.dm.position()
        
        # ALSO you can get secondary positions 
        # secondary_position = self.dm.position('CONTFUT')       

        # get net exposure for all members
        # exposure_record - is a slice of exposures results of picked alphas at 'date' 
        
        # We are calling sum() because we have multiple records of 'exposure'
        # 1-alpha member of best in the swarm per row
        exposure = exposure_record['exposure'].sum()

        #
        # Just replicate primary quotes position
        #
        replicated_pos = primary_quotes_position.get_net_position(date)
        self.position.add_net_position(date, replicated_pos, qty=exposure)

# Alpha load/save/run

** Simple steps to run/load/save alphas **

## Step 1: Load modules

In [ ]:
from tmqrfeed.manager import DataManager
from tmqrstrategy.optimizers import OptimizerBase, OptimizerGenetic

#
# IMPORTANT! To save alpha you must deploy it as stand-alone module
#            saving of alphas from notebooks in not allowed!
#

# Load deployed alpha module
from tmqrstrategy.tests.debug_alpha_prototype import AlphaGeneric

## Step 2: Set the context and params

**WARNING** You should set the context only once at the first time alpha is deployed, then the context will 
be loaded automatically on each alpha.load() call.

So you need set all of the parameters only once, if you would like to change params of deployed alpha you should rerun this alpha ommitting alpha.load() call. 

**NOTE** Due to stochastic (random) nature of some algorithms (like ML or GeneticOptimizer), there is no guarantee that you will get same results. It's very dangerous to rewrite and rerun alphas which are already deployed and in the production, better to create new version of alpha and gracefully disengage old version.

### Alpha context settings how to

### General settings
    'name': 'ES_NewFramework_MACross_Genetic', # Global alpha name, which be used for load/save to DB
    
    'context': { # Strategy specific settings
            # These settings only applycable for alphas derived from StrategyAlpha strategy 
            # StrategyAlpha - is a classic EXO/SmartEXO based alpha
            'index_name': 'US.ES_ContFutEOD',      # Name of EXO index to trade
            'costs_per_option': 3.0,
            'costs_per_contract': 3.0,
        },

### Walk-forward optimization parameters:

    'wfo_params': {
            'window_type': 'rolling',  # Rolling window for IIS values: rolling or expanding
            'period': 'M',             # Period of rolling window 'M' - monthly or 'W' - weekly
            'oos_periods': 2,          # Number of months is OOS period
            'iis_periods': 12,         # Number of months in IIS rolling window and minimal test period len
                                       # (only applicable for 'window_type' == 'rolling')
        },

### Optimizer class parameters
By design alpha can use any optimization algorithm, OptimizerClass permutates 'opt_params' and calculate alphas using these params, then it select best alphas by alpha.score() method results, and finally call alpha.pick() to 
select best performing alphas for each WFO step.

'optimizer_class_kwargs' - OptimizerClass parameters, refer to source code to get more info.


        'wfo_optimizer_class': OptimizerGenetic,
        'wfo_optimizer_class_kwargs': {
            'nbest_count': 3,
            'nbest_fitness_method': 'max',
            'population_size': 50, 
            'number_generations': 30, 
            # 'rand_seed': 1, # Uncomment this parameter to make genetic results repeatable
        },

### Alpha's optimization parameters
The order of 'opt_params' list should be the same as arguments order in alpha.calculate() method for particular alpha.

        'wfo_opt_params': [
            ('period_slow', [10, 30, 40, 50, 70, 90, 110]),
            ('period_fast', [1, 3, 10, 15, 20, 30])
        ],

### WFO Scoring functions params
- 'wfo_members_count' - number of picked alphas at each out-of-sample WFO step
- 'wfo_costs_per_contract' - costs in USD per contract used in WFO scoring functions (used only for alphas picking!, you should set costs explicitly for each alpha in the alpha.setup() method)
- 'wfo_scoring_type' - type of scoring function to rank alphas on in-sample period of WFO


        'wfo_members_count': 1,
        'wfo_costs_per_contract': 0.0,
        'wfo_scoring_type': 'netprofit'

In [ ]:
ALPHA_CONTEXT = {
        'name': 'ES_NewFramework_MACross_Genetic', # Global alpha name, which be used for load/save from DB
        'context': { # Strategy specific settings
            # These settings only applycable for alphas derived from StrategyAlpha strategy 
            # StrategyAlpha - is a classic EXO/SmartEXO based alpha
            'index_name': 'US.ES_ContFutEOD',      # Name of EXO index to trade
            'costs_per_option': 3.0,
            'costs_per_contract': 3.0,
        },
        'wfo_params': {
            'window_type': 'rolling',  # Rolling window for IIS values: rolling or expanding
            'period': 'M',  # Period of rolling window 'M' - monthly or 'W' - weekly
            'oos_periods': 2,  # Number of months is OOS period
            'iis_periods': 12,
            # Number of months in IIS rolling window (only applicable for 'window_type' == 'rolling')
        },
        'wfo_optimizer_class': OptimizerBase,
        'wfo_optimizer_class_kwargs': {
            'nbest_count': 3,
            'nbest_fitness_method': 'max'
        },
        'wfo_opt_params': [
            ('period_slow', [10, 30, 40, 50, 70, 90, 110]),
            ('period_fast', [1, 3, 10, 15, 20, 30])
        ],
        'wfo_members_count': 1,
        'wfo_costs_per_contract': 0.0,
        'wfo_scoring_type': 'netprofit'
    }

### Step 3: Init and run alpha (first run / development / first deployment case)

In [ ]:
# DataManager is a core class of the framework
dm = DataManager()

# Init alpha class and run
alpha = AlphaGeneric(dm, **ALPHA_CONTEXT)    

In [ ]:
# Run alpha's WFO optimization from scratch
alpha.run()

## Step 4: now we are ready to save

In [ ]:
alpha.save()

print(f"Saved alpha with name: {ALPHA_CONTEXT['name']}")

## Step 5: Ready for online use

In [ ]:
# Load deployed alpha module
from tmqrstrategy.tests.debug_alpha_prototype import AlphaGeneric

# Or load Strategy base
from tmqrstrategy import StrategyBase

In [ ]:
# Init the environment
dm2 = DataManager()

# Do first run
alpha_name = ALPHA_CONTEXT['name']

# Call <AlphaClass>.load(datamanager, alpha_name)
saved_alpha = AlphaGeneric.load(dm2, alpha_name)

# BOTH METHODS ARE EQUAL!

# Call StrategyBase.load(datamanager, alpha_name)
# StrategyBase - can be more usefun in online scripts
saved_alpha = StrategyBase.load(dm2, alpha_name)

# The alpha.run() - only calculate recent data, and do another WFO step if required
saved_alpha.run()

# Save it again!
saved_alpha.save()


#
# Finally you are ready to process alpha's positions for campaings!
#

# Alpha information

After alpha has been run, you can get some information about the process.

## Alpha equity and position

### Option 1: generate equity from alpha's position (dynamicaly)

In [ ]:
equity_data = alpha.position.get_pnl_series()

In [ ]:
equity_data.tail()

In [ ]:
# Equity based on decision price
equity_data['equity_decision'].plot();

In [ ]:
# Equity based on execution price
equity_data['equity_execution'].plot();

### Option 2: get equity from stats (statically - i.e. this field saved in the DB)

In [ ]:
alpha.stats['series'].tail()

In [ ]:
alpha.stats['series']['equity'].plot();

## Alpha position fetching

In [ ]:
alpha.position

In [ ]:
position_last_date = alpha.position.last_date
print(position_last_date)

In [ ]:
# Delta of the alpha at last date 
alpha.position.delta(position_last_date)

## Alpha exposure

This a historical series of 'exposure' dataframe returned by calculate only at out-of-sample step of WFO. 

In [ ]:
alpha.exposure_series.tail()